<br>
<img align="center" width="500" src="rflogo.png">
<br>

# Optimizing the Tenant Mix
<br>
<img align="left" width="80" height="200" src="https://img.shields.io/badge/python-v3.6-blue.svg">
<br>

<br>

### Notebook by [Marco Tavora](https://marcotavora.me/)

## Table of contents
 
1. [Import Modules](#Import-Modules)
2. [Problem](#Problem)
3. [Cleaning up](#Cleaning-up)
4. [Dictionary from codes to variables](#Dictionary-from-codes-to-variables)
5. [Defining dimensions](#Defining-dimensions)
6. [Generating artificial parameters to build code](#Generating-artificial-parameters-to-build-code)
7. [Code](#Code)

### Import Modules  
[[go back to the top]](#Table-of-contents)

In [1]:
from pulp import *

import matplotlib.pyplot as plt
import random
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
plt.style.use('seaborn-whitegrid')
import numpy as np
import string, time
import math as m
import warnings      
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

import unidecode
def un_st(x):
    return unidecode.unidecode(x)

### Problem 
[[go back to the top]](#Table-of-contents)

The objective function to be maximized is a sum of two terms:
- The present worth (or value) of all stores $(i,j,k)$
- The total marginal revenue obtained by including the $l$-th type-$i$ store when there are at least $l$ stores of type $i$. 

More formally the objective function is:

$$f = \,\,\sum\limits_{ijkl} {({\rm{P}}{{\rm{W}}_{ijkl}}{x_{ijk}} + {R_{il}}{S_{il}})\,} $$

and the goal is to solve:

$$\mathop {\max }\limits_{{x_{ijk}}} f=\mathop {\max }\limits_{{x_{ijk}}} \,\,\sum\limits_{ijkl} {({\rm{P}}{{\rm{W}}_{ijkl}}{x_{ijk}} + {R_{il}}{S_{il}})\,} $$

given a set of constraints given in the appendix of this [article](https://pubsonline.informs.org/doi/abs/10.1287/inte.18.2.1) by Bean *et al* and which I will not reproduce here.

Let us consider an example for concreteness. Suppose a given store $(i,j,k)\equiv(1,1,1)$ is a shoes store (store type $i\equiv 1$), located in a central walkway between two anchors (location class $j\equiv1$), and with 200 square feet (size $k\equiv 1$) of area. Furthermore, suppose there are $l=10$ shoes stores (include the $i=1$ store). For clarity of exposition suppose that, when the store started its activities, there were already 9 shoes stores at the mall. We then denote the revenue added to the present value $f$ of including this 10th store to be $R_{1,10}$. The dummy variable $S_{1,10}=1$ since there are 10 stores of type $i=1$. 

### Cleaning up
[[go back to the top]](#Table-of-contents)

In [2]:
file = 'shopping_ficticio_v6.xlsx'
xl = pd.ExcelFile(file)
print('sheet names:',xl.sheet_names)
(shopping, lojas, varej) = (xl.parse(xl.sheet_names[0]), 
                           xl.parse(xl.sheet_names[1], skiprows=1), 
                           xl.parse(xl.sheet_names[2]))

sheet names: ['Shopping', 'Lojas', 'Varejistas']


In [3]:
lojas.head(2)

,Loja,Piso,Corredor (J = 3 (location classes)),Tipo de Loja,Loja Esquerda,Loja Direita,Loja em Frente,Área (m2),K = 5 (store size classes),Vitrine (m),Esquina (Sim/Não),Ocupante,Segmento (I = 21 (store types)),Tipo de Segmento,Faturamento Mensal,Ticket Médio,Conv. Corredor/Loja,Conv. Loja/Compras,Fluxo Diário no Corredor,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,1,1,Corredor 01,Satélite,Entrada 01 Esquerda,3,106,294.0,4.0,12.40,Não,Ótica São Pedro,Ótica,Impulso,65610.0,45.0,0.05,0.15,6480.0,NaN,NaN,NaN,NaN,NaN,Store size class,Store size class,Store size class,Store size class,Store size class,Location class,Location class,Location class
1,2,1,Corredor 03,Alimentação,84,85,71,44.0,1.0,5.68,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Store Types,Num Lojas,Total ABL (m2),1,2,3,4,5,Corredor 01,Corredor 02,Corredor 03


In [4]:
nan_lst = []
n = 5
for s in lojas.columns:
    lst = [s, round(lojas[s].isna().sum()/lojas.shape[0],2)]
    if lst[1] == 1.0:
        nan_lst.append(lst)
cols_to_drop = [el[0] for el in nan_lst] + ['Unnamed: 20']
lojas = lojas.drop(cols_to_drop, axis=1)
lojas.head(2)

,Loja,Piso,Corredor (J = 3 (location classes)),Tipo de Loja,Loja Esquerda,Loja Direita,Loja em Frente,Área (m2),K = 5 (store size classes),Vitrine (m),Esquina (Sim/Não),Ocupante,Segmento (I = 21 (store types)),Tipo de Segmento,Faturamento Mensal,Ticket Médio,Conv. Corredor/Loja,Conv. Loja/Compras,Fluxo Diário no Corredor,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,1,1,Corredor 01,Satélite,Entrada 01 Esquerda,3,106,294.0,4.0,12.40,Não,Ótica São Pedro,Ótica,Impulso,65610.0,45.0,0.05,0.15,6480.0,NaN,NaN,NaN,Store size class,Store size class,Store size class,Store size class,Store size class,Location class,Location class,Location class
1,2,1,Corredor 03,Alimentação,84,85,71,44.0,1.0,5.68,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Store Types,Num Lojas,Total ABL (m2),1,2,3,4,5,Corredor 01,Corredor 02,Corredor 03


#### Tabela 1

In [5]:
col_to_crop = lojas.columns.tolist().index('Unnamed: 21')
lojas_1 = lojas.iloc[:,:col_to_crop]
lojas_1.dropna(inplace=True)
lojas_1.columns = [un_st(x.lower().replace('(','').replace(')',''))
                    for x in lojas_1.columns]
lojas_1.head(2)

,loja,piso,corredor j = 3 location classes,tipo de loja,loja esquerda,loja direita,loja em frente,area m2,k = 5 store size classes,vitrine m,esquina sim/nao,ocupante,segmento i = 21 store types,tipo de segmento,faturamento mensal,ticket medio,conv. corredor/loja,conv. loja/compras,fluxo diario no corredor
0,1,1,Corredor 01,Satélite,Entrada 01 Esquerda,3,106,294.0,4.0,12.4,Não,Ótica São Pedro,Ótica,Impulso,65610.0,45.0,0.05,0.150,6480.0
2,3,1,Corredor 01,Satélite,1,11,105,40.0,1.0,6.0,Não,CVC,Serviço,Serviço,218700.0,750.0,0.02,0.075,6480.0


In [6]:
lojas_1.isnull().any().unique()

array([False])

#### Tabela 2

In [7]:
lojas_2 = lojas.iloc[:, col_to_crop:]
lojas_2.iloc[1,:] = ['Store Types', 'Num Lojas', 'Total ABL (m2)', 'Store size class_1', 'Store size class_2',
                   'Store size class_3', 'Store size class_4', 'Store size class_5', 
                   'Location class Corredor 01', 'Location class Corredor 02', 'Location class Corredor 03']
lojas_2 = lojas_2.iloc[1:,:]
lojas_2.columns = lojas_2.iloc[0,:]
lojas_2 = lojas_2.iloc[1:22, :]
lojas_1.dropna(inplace=True)
lojas_2.columns = [un_st(x.lower().replace(' ','_').replace('(','').replace(')',''))
                    for x in lojas_2.columns]
lojas_2['store_types'] = [un_st(x.lower().replace('.', ' ').replace('(','').replace(')','')) 
                          for x in list(lojas_2['store_types'])]

In [8]:
lojas_2.columns = ['store_types', 'num_lojas', 'total_abl_m2', 'size_1', 'size_2', 'size_3', 'size_4', 'size_5',
           'corredor_01', 'corredor_02', 'corredor_03']

In [9]:
lojas_2.head()

,store_types,num_lojas,total_abl_m2,size_1,size_2,size_3,size_4,size_5,corredor_01,corredor_02,corredor_03
2,acessorios femininos,4,273,1,3,0,0,0,3,1,0
3,alimentacao,16,1154,7,7,2,0,0,0,0,16
4,art esportivos,1,679,0,0,0,0,1,1,0,0
5,brinquedos,1,72,0,1,0,0,0,0,0,1
6,cafeteria,2,115,1,1,0,0,0,1,1,0


In [10]:
lojas_2.isnull().any().unique()

array([False])

In [17]:
types_original = list(lojas_2['store_types'])
lst = []
for x in list(range(1,len(types_original)+1)):
    lst.append('t'+ str(x))
types = lst
data = dict(zip(types_original, lst))
types_df = pd.DataFrame(list(data.items()), 
                        columns=['type_code', 'type'])

locations_original = ['corredor_01', 'corredor_02', 'corredor_03']
locations_df = pd.DataFrame(list(dict(zip(locations_original, ['c1', 'c2', 'c3'])).items()), 
                            columns=['location_code', 'location'])
sizes_original = ['size_1', 'size_2', 'size_3', 'size_4', 'size_5']
sizes_df = pd.DataFrame(list(dict(zip(sizes_original, ['s1', 's2', 's3', 's4', 's5'])).items()), 
                            columns=['size_code', 'size'])

### Dictionary from codes to variables
[[go back to the top]](#Table-of-contents)

In [18]:
print('type codes:')
types_df
print('location codes:')
locations_df
print('size codes:')
sizes_df

type codes:


,type_code,type
0,acessorios femininos,t1
1,alimentacao,t2
2,art esportivos,t3
3,brinquedos,t4
4,cafeteria,t5
5,calcados,t6
6,cama e banho,t7
7,colchoes,t8
8,departamento,t9
9,drogaria,t10


location codes:


,location_code,location
0,corredor_01,c1
1,corredor_02,c2
2,corredor_03,c3


size codes:


,size_code,size
0,size_1,s1
1,size_2,s2
2,size_3,s3
3,size_4,s4
4,size_5,s5


### Defining dimensions
[[go back to the top]](#Table-of-contents)

In [19]:
I, J, K = len(types), len(locations), len(sizes)

stores = [(i, j, k) for i in range(I) 
          for j in range(J) 
          for k in range(K)]

print('number of x:', len(stores))
for x in [[I, 'types'], [J, 'locations'],[K, 'sizes']]:
    print('number of {} classes is:'.format(x[1]),'is', x[0])

number of x: 315
number of types classes is: is 21
number of locations classes is: is 3
number of sizes classes is: is 5


### Generating artificial parameters to build code
[[go back to the top]](#Table-of-contents)

For simplicity we choose $I,J$ and $K$ equal to 2. But **the code is valid for any values of I, J, K**.

The appropriate values of

    (FW, A, G, F, f, L, M, m, R, S, B, NS)
    
must be chosen. 

\begin{eqnarray}
&&A_{ik} = \text{amount of area required for a store of type } i\,\, \text{and size} \,\,k.\nonumber\\
&&G_{j} = \text{total amount of square feet (gross leaseable area) available in location class} \,\,j\nonumber\\
&&f_i = \text{least amount of square feet available for type} \,\,i\nonumber\\
&&F_i = \text{largest amount of square feet available for type} \,\,i \nonumber\\
&&L_i = \text{amount of finishing allowance given to a tenant of type} \,\,i\,\, \text{per square foot leased}.\nonumber\\
&&B = \text{tenant allowance budget}\nonumber\\
&&M_i = \text{maximum number of tenants of type} \,\,i.\nonumber\\
&&m_i = \text{minimum number of tenants of type} \,\,i.\nonumber\\
&&N_s = \text{maximum number of small stores}\nonumber\\
&&S_{il} = \text{binary variable set to 1 if there are at least} \,\,l \,\,\text{stores of type} \,\,i\nonumber\\
&& {\text{PW}}_{ijkl} = \text{present worth of a store de scribed by} \,\,(i, j, k) \,\,\text{if it is one of} \,\,l \,\,\text{stores of type} \,\,i\nonumber\\
&& R_{il} = \text{average marginal revenue added by including the}\,\, l\text{-th} \,\,\text{store of type}\,\, i\nonumber\\
\end{eqnarray}




In [27]:
(upBound, I, J, K, FW, A, G, F, f, L, M, m, R, S, B, NS) = (len(stores), 2, 2, 2, 
                                                              {(0, 0, 0): 655, (0, 0, 1): 115, (0, 1, 0): 26, 
                                                               (0, 1, 1): 760, (1, 0, 0): 282, (1, 0, 1): 251, 
                                                               (1, 1, 0): 229, (1, 1, 1): 143}, 
                                                              {(0, 0): 54, (0, 1): 275, (1, 0): 90, (1, 1): 788}, 
                                                              {0: 351, 1: 272}, 
                                                              {0: 790, 1: 292}, 
                                                              {0: 789, 1: 291}, 
                                                              {0: 730, 1: 393}, 
                                                              {0: 730, 1: 393}, 
                                                              {0: 729, 1: 392},
                                                              {(0, 729): 0, (0, 730): 0, (1, 392): 392, (1, 393): 393},
                                                              {(0, 729): 1458, (0, 730): 1460, (1, 392): 785, (1, 393): 787},
                                                              100, 
                                                              100)

#### Equations from the paper

\[\begin{array}{l}
{\text{total income}} = \sum\limits_{i = 1}^I {\sum\limits_{l = {m_i}}^{{M_i}} {{R_{il}}{S_{il}}} }
\end{array}\]

In [25]:
total_income = 0
for i in range(I):
    total_income += sum([R[i, l] * S[i, l] for l in range(m[i], M[i]+1)])

\[\begin{array}{l}
{({\rm{dict\_S)}}_i}{\rm{ }} = \,\sum\limits_{l = {m_i}}^{{M_i}} {{S_{il}}} 
\end{array}\]

In [26]:
dict_S = {}
for i in range(I):
    dict_S[i] = sum([S[i, l] for l in range(m[i], M[i]+1)])

### Code
[[go back to the top]](#Table-of-contents)

In [28]:
def tenant_opt(upBound, I, J, K, FW, A, G, F, f, dict_S, L, M, m, B, NS):

    x = pulp.LpVariable.dicts('x', stores, lowBound = 0, upBound = upBound, cat = pulp.LpInteger)

    prob1 = pulp.LpProblem('TenantOptimization', LpMaximize)
    prob1 += sum([x[i, j, k] * FW[i, j, k] 
                  + total_income/(I*J*K) 
                  for i in range(I) 
                  for j in range(J) 
                  for k in range(K)])

    for j in range(J):
        prob1 += lpSum([A[i, k] * x[i, j, k] for i in range(I) for k in range(K)]) <= G[j]

    for i in range(I):
        prob1 += lpSum([A[i, k] * x[i, j, k] for j in range(J) for k in range(K)]) <= F[i]  
        prob1 += lpSum([A[i, k] * x[i, j, k] for j in range(J) for k in range(K)]) >= f[i] 
        prob1 += lpSum([x[i, j, k] for j in range(J) for k in range(K)]) <= M[i]  
        prob1 += lpSum([x[i, j, k] for j in range(J) for k in range(K)]) >= m[i] 
        prob1 += lpSum([x[i, j, k] for i in range(I) for j in range(J) for k in range(K)]) == dict_S[i]
        prob1 += dict_S[i] == dict_S[i]

    prob1 += lpSum([x[i, j, 1] for i in range(I) for j in range(J)]) <= NS  
    prob1 += lpSum([L[i] * A[i, k] * x[i, j, k] for i in range(I) for j in range(J) for k in range(K)]) <= B
    
    return prob1

tenant_opt(upBound, I, J, K, FW, A, G, F, f, dict_S, L, M, m, B, NS)

TenantOptimization:
MAXIMIZE
655*x_(0,_0,_0) + 115*x_(0,_0,_1) + 26*x_(0,_1,_0) + 760*x_(0,_1,_1) + 282*x_(1,_0,_0) + 251*x_(1,_0,_1) + 229*x_(1,_1,_0) + 143*x_(1,_1,_1) + 617011.0
SUBJECT TO
_C1: 54 x_(0,_0,_0) + 275 x_(0,_0,_1) + 90 x_(1,_0,_0) + 788 x_(1,_0,_1)
 <= 351

_C2: 54 x_(0,_1,_0) + 275 x_(0,_1,_1) + 90 x_(1,_1,_0) + 788 x_(1,_1,_1)
 <= 272

_C3: 54 x_(0,_0,_0) + 275 x_(0,_0,_1) + 54 x_(0,_1,_0) + 275 x_(0,_1,_1)
 <= 790

_C4: 54 x_(0,_0,_0) + 275 x_(0,_0,_1) + 54 x_(0,_1,_0) + 275 x_(0,_1,_1)
 >= 789

_C5: x_(0,_0,_0) + x_(0,_0,_1) + x_(0,_1,_0) + x_(0,_1,_1) <= 730

_C6: x_(0,_0,_0) + x_(0,_0,_1) + x_(0,_1,_0) + x_(0,_1,_1) >= 729

_C7: x_(0,_0,_0) + x_(0,_0,_1) + x_(0,_1,_0) + x_(0,_1,_1) + x_(1,_0,_0)
 + x_(1,_0,_1) + x_(1,_1,_0) + x_(1,_1,_1) = 2918

_C8: 90 x_(1,_0,_0) + 788 x_(1,_0,_1) + 90 x_(1,_1,_0) + 788 x_(1,_1,_1)
 <= 292

_C9: 90 x_(1,_0,_0) + 788 x_(1,_0,_1) + 90 x_(1,_1,_0) + 788 x_(1,_1,_1)
 >= 291

_C10: x_(1,_0,_0) + x_(1,_0,_1) + x_(1,_1,_0) + x_(1,_1,_1